In [1]:
import pickle as pkl
import xgboost as xgb
import gzip
import os
import fnmatch

import json

In [2]:
weightDir = '/work/gallim/weights/2018_MIX/good_finals'

In [3]:
def dump_xgb(pkl_fname,weightsDir,outDir):
    clf = pkl.load(gzip.open('{}/{}'.format(weightsDir,pkl_fname)))['clf']
    booster = clf.get_booster()
    var = pkl_fname[pkl_fname.index('probe')+5:pkl_fname.index('.')]
    print(var)
    outfName = pkl_fname[:pkl_fname.index('.')].replace('probe','').replace(var,replacement[var])
    print(outfName)
    booster.save_model('{}/{}.xgb'.format(outDir,outfName))

def dump_xgb_text(pkl_fname,weightsDir,outDir):
    clf = pkl.load(gzip.open('{}/{}'.format(weightsDir,pkl_fname)))['clf']
    booster = clf.get_booster()
    booster.dump_model('{}/{}.txt'.format(outDir,pkl_fname[:pkl_fname.index('.')].replace("probe","")))

In [4]:
replacement = {'R9': 'r9', 'S4': 's4', 'CovarianceIeIp': 'sieip', 'SigmaIeIe': 'sieie', 'PhiWidth': 'phiWidth', 'EtaWidth': 'etaWidth', 'PhoIso': 'phoIso', 'ChIso03worst': 'chIsoWorst','ChIso03': 'chIso'}

In [5]:
for file in os.listdir(weightDir):
    if fnmatch.fnmatch(file,'weights_*.pkl'):
#         dump_xgb_text(file,weightDir,'/t3home/threiten/OLD-HOME/QReg/ReReco17_data/camp_3_1_0/probePS/fullSample/weights_xgb/')
        dump_xgb(file,weightDir,'{}/xgb'.format(weightDir))

S4
weights_finalRegressor_EB_s4
PhiWidth
weights_finalRegressor_EE_phiWidth
ChIso03worst
weights_finalRegressor_EB_chIsoWorst
ChIso03
weights_finalRegressor_EB_chIso
EtaWidth
weights_finalRegressor_EB_etaWidth
R9
weights_finalRegressor_EE_r9
PhoIso
weights_finalTailRegressor_EE_phoIso
ChIso03worst
weights_finalRegressor_EE_chIsoWorst
PhoIso
weights_finalRegressor_EE_phoIso
EtaWidth
weights_finalRegressor_EE_etaWidth
PhiWidth
weights_finalRegressor_EB_phiWidth
ChIso03
weights_finalRegressor_EE_chIso
ChIso03
weights_finalTailRegressor_EB_chIso
R9
weights_finalRegressor_EB_r9
CovarianceIeIp
weights_finalRegressor_EE_sieip
PhoIso
weights_finalRegressor_EB_phoIso
S4
weights_finalRegressor_EE_s4
ChIso03worst
weights_finalTailRegressor_EB_chIsoWorst
SigmaIeIe
weights_finalRegressor_EB_sieie
SigmaIeIe
weights_finalRegressor_EE_sieie
ChIso03
weights_finalTailRegressor_EE_chIso
PhoIso
weights_finalTailRegressor_EB_phoIso
CovarianceIeIp
weights_finalRegressor_EB_sieip
ChIso03worst
weights_finalTa

In [6]:
year = '2018_Legacy'
dic_SS={}
for var in ['probeR9','probeS4','probeSigmaIeIe','probePhiWidth','probeEtaWidth','probeCovarianceIeIp']:
    scalerEB = pkl.load(gzip.open('{}/scaler_mc_EB_{}_corr_diff.pkl'.format(weightDir,var)))
    scalerEE = pkl.load(gzip.open('{}/scaler_mc_EE_{}_corr_diff.pkl'.format(weightDir,var)))
    varOut = replacement[var.replace('probe','')]
    dic_SS[varOut]={}
    dic_SS[varOut]['EB']={}
    dic_SS[varOut]['EB']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EB_{}.xml".format(year, varOut)
    dic_SS[varOut]['EB']['scale'] = scalerEB.scale_[0]
    dic_SS[varOut]['EB']['center'] = scalerEB.center_[0]
    dic_SS[varOut]['EE']={}
    dic_SS[varOut]['EE']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EE_{}.xml".format(year, varOut)
    dic_SS[varOut]['EE']['scale'] = scalerEE.scale_[0]
    dic_SS[varOut]['EE']['center'] = scalerEE.center_[0]
    
dic_phoIso = {}
for var in ['probePhoIso']:
    scalerEB = pkl.load(gzip.open('{}/scaler_mc_EB_{}_corr_diff.pkl'.format(weightDir,var)))
    scalerEE = pkl.load(gzip.open('{}/scaler_mc_EE_{}_corr_diff.pkl'.format(weightDir,var)))
    varOut = replacement[var.replace('probe','')]
    dic_phoIso['EE'] = {}
    dic_phoIso['EE']['peak_tail_clfs'] = {}
    dic_phoIso['EE']['peak_tail_clfs']['weights_data'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/data_clf_p2t_EE_{}.xml".format(year, varOut)
    dic_phoIso['EE']['peak_tail_clfs']['weights_mc'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/mc_clf_p2t_EE_{}.xml".format(year, varOut)
    dic_phoIso['EE']['peak_tail_clfs']['variables'] = [ "f0 := pt",                                                                                                 
                                                        "f1 := superCluster.eta",                                                                                   
                                                        "f2 := phi",                                                                                                
                                                        "f3 := global.rho"]
    
    dic_phoIso['EE']['peak2tail'] = {}
    dic_phoIso['EE']['peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EE_{}.xml".format(year, varOut)
    dic_phoIso['EE']['peak2tail']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := userFloat(\"peak2tail_rnd\")"]
    
    dic_phoIso['EE']['morphing'] = {}
    dic_phoIso['EE']['morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EE_{}.xml".format(year, varOut)
    dic_phoIso['EE']['morphing']['scale'] = scalerEE.scale_[0]
    dic_phoIso['EE']['morphing']['center'] = scalerEE.center_[0]
    dic_phoIso['EE']['morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfPhoIso03"]
    
    dic_phoIso['EB'] = {}
    dic_phoIso['EB']['peak_tail_clfs'] = {}
    dic_phoIso['EB']['peak_tail_clfs']['weights_data'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/data_clf_p2t_EB_{}.xml".format(year, varOut)
    dic_phoIso['EB']['peak_tail_clfs']['weights_mc'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/mc_clf_p2t_EB_{}.xml".format(year, varOut)
    dic_phoIso['EB']['peak_tail_clfs']['variables'] = [ "f0 := pt",                                                                                                 
                                                        "f1 := superCluster.eta",                                                                                   
                                                        "f2 := phi",                                                                                                
                                                        "f3 := global.rho"]
    
    dic_phoIso['EB']['peak2tail'] = {}
    dic_phoIso['EB']['peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EB_{}.xml".format(year, varOut)
    dic_phoIso['EB']['peak2tail']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := userFloat(\"peak2tail_rnd\")"]
    
    dic_phoIso['EB']['morphing'] = {}
    dic_phoIso['EB']['morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EB_{}.xml".format(year, varOut)
    dic_phoIso['EB']['morphing']['scale'] = scalerEB.scale_[0]
    dic_phoIso['EB']['morphing']['center'] = scalerEB.center_[0]
    dic_phoIso['EB']['morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfPhoIso03"]
    

dic_chIso = {}

scalerEB_chIso = pkl.load(gzip.open('{}/scaler_mc_EB_probeChIso03_corr_diff.pkl'.format(weightDir)))
scalerEE_chIso = pkl.load(gzip.open('{}/scaler_mc_EE_probeChIso03_corr_diff.pkl'.format(weightDir)))
scalerEB_chIsoWorst = pkl.load(gzip.open('{}/scaler_mc_EB_probeChIso03worst_corr_diff.pkl'.format(weightDir)))
scalerEE_chIsoWorst = pkl.load(gzip.open('{}/scaler_mc_EE_probeChIso03worst_corr_diff.pkl'.format(weightDir)))
# varOut = replacement[var.replace('probe','')]

dic_chIso['EE'] = {}
dic_chIso['EE']['peak_tail_clfs'] = {}
dic_chIso['EE']['peak_tail_clfs']['weights_data'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/data_clf_3Cat_EE_chIso_chIsoWorst.xml".format(year)
dic_chIso['EE']['peak_tail_clfs']['weights_mc'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/mc_clf_3Cat_EE_chIso_chIsoWorst.xml".format(year)
dic_chIso['EE']['peak_tail_clfs']['variables'] = ["f0 := pt",                                                                                                 
                                                  "f1 := superCluster.eta",                                                                                   
                                                  "f2 := phi",                                                                                                
                                                  "f3 := global.rho"]

dic_chIso['EE']['chIso_peak2tail'] = {}
dic_chIso['EE']['chIso_peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EE_chIso.xml".format(year)
dic_chIso['EE']['chIso_peak2tail']['variables'] = ["f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtWorstVtx03",
                                                   "f5 := userFloat(\"peak2tail_chIso_rnd\")"]

dic_chIso['EE']['chIsoWorst_peak2tail'] = {}
dic_chIso['EE']['chIsoWorst_peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EE_chIsoWorst.xml".format(year)
dic_chIso['EE']['chIsoWorst_peak2tail']['variables'] = ["f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtChosenVtx03",
                                                   "f5 := userFloat(\"peak2tail_chIsoWorst_rnd\")"]

dic_chIso['EE']['chIso_morphing'] = {}
dic_chIso['EE']['chIso_morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EE_chIso.xml".format(year)
dic_chIso['EE']['chIso_morphing']['scale'] = scalerEE_chIso.scale_[0]
dic_chIso['EE']['chIso_morphing']['center'] = scalerEE_chIso.center_[0]
dic_chIso['EE']['chIso_morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtChosenVtx03",
                                                   "f5 := pfChgIsoWrtWorstVtx03"]

dic_chIso['EE']['chIsoWorst_morphing'] = {}
dic_chIso['EE']['chIsoWorst_morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EE_chIsoWorst.xml".format(year)
dic_chIso['EE']['chIsoWorst_morphing']['scale'] = scalerEE_chIsoWorst.scale_[0]
dic_chIso['EE']['chIsoWorst_morphing']['center'] = scalerEE_chIsoWorst.center_[0]
dic_chIso['EE']['chIsoWorst_morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                             "f1 := superCluster.eta",                                                                                   
                                                             "f2 := phi",                                                                                                
                                                             "f3 := global.rho",
                                                             "f4 := pfChgIsoWrtChosenVtx03",
                                                             "f5 := pfChgIsoWrtWorstVtx03"]

dic_chIso['EB'] = {}
dic_chIso['EB']['peak_tail_clfs'] = {}
dic_chIso['EB']['peak_tail_clfs']['weights_data'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/data_clf_3Cat_EB_chIso_chIsoWorst.xml".format(year)
dic_chIso['EB']['peak_tail_clfs']['weights_mc'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/mc_clf_3Cat_EB_chIso_chIsoWorst.xml".format(year)
dic_chIso['EB']['peak_tail_clfs']['variables'] = ["f0 := pt",                                                                                                 
                                                  "f1 := superCluster.eta",                                                                                   
                                                  "f2 := phi",                                                                                                
                                                  "f3 := global.rho"]

dic_chIso['EB']['chIso_peak2tail'] = {}
dic_chIso['EB']['chIso_peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EB_chIso.xml".format(year)
dic_chIso['EB']['chIso_peak2tail']['variables'] = ["f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtWorstVtx03",
                                                   "f5 := userFloat(\"peak2tail_chIso_rnd\")"]

dic_chIso['EB']['chIsoWorst_peak2tail'] = {}
dic_chIso['EB']['chIsoWorst_peak2tail']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalTailRegressor_EB_chIsoWorst.xml".format(year)
dic_chIso['EB']['chIsoWorst_peak2tail']['variables'] = ["f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtChosenVtx03",
                                                   "f5 := userFloat(\"peak2tail_chIsoWorst_rnd\")"]

dic_chIso['EB']['chIso_morphing'] = {}
dic_chIso['EB']['chIso_morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EB_chIso.xml".format(year)
dic_chIso['EB']['chIso_morphing']['scale'] = scalerEB_chIso.scale_[0]
dic_chIso['EB']['chIso_morphing']['center'] = scalerEB_chIso.center_[0]
dic_chIso['EB']['chIso_morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                   "f1 := superCluster.eta",                                                                                   
                                                   "f2 := phi",                                                                                                
                                                   "f3 := global.rho",
                                                   "f4 := pfChgIsoWrtChosenVtx03",
                                                   "f5 := pfChgIsoWrtWorstVtx03"]

dic_chIso['EB']['chIsoWorst_morphing'] = {}
dic_chIso['EB']['chIsoWorst_morphing']['weights'] = "flashgg/Taggers/data/PhoIdInputsCorrections/{}/weights_finalRegressor_EB_chIsoWorst.xml".format(year)
dic_chIso['EB']['chIsoWorst_morphing']['scale'] = scalerEB_chIsoWorst.scale_[0]
dic_chIso['EB']['chIsoWorst_morphing']['center'] = scalerEB_chIsoWorst.center_[0]
dic_chIso['EB']['chIsoWorst_morphing']['variables'] = [ "f0 := pt",                                                                                                 
                                                             "f1 := superCluster.eta",                                                                                   
                                                             "f2 := phi",                                                                                                
                                                             "f3 := global.rho",
                                                             "f4 := pfChgIsoWrtChosenVtx03",
                                                             "f5 := pfChgIsoWrtWorstVtx03"]

In [7]:
print(dic_SS)
print('------------------------------------------------------------------------------------------------------------\n')
print(dic_phoIso)
print('------------------------------------------------------------------------------------------------------------\n')
print(dic_chIso)

{'r9': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_r9.xml', 'scale': 0.00974529926595541, 'center': -0.0032455444688871404}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EE_r9.xml', 'scale': 0.015158424850746033, 'center': -0.008216661026002492}}, 's4': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_s4.xml', 'scale': 0.006867933306285823, 'center': -0.002979937739167826}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EE_s4.xml', 'scale': 0.013772174062689158, 'center': -0.01275499890321552}}, 'sieie': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_sieie.xml', 'scale': 9.465761735043482e-05, 'center': -1.3066554941611706e-05}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegre

In [8]:
dic = {}
dic['shower_shapes'] = dic_SS
dic['isolations'] = {}
dic['isolations']['phoIso'] = dic_phoIso
dic['isolations']['chIso'] = dic_chIso
print(dic)

{'shower_shapes': {'r9': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_r9.xml', 'scale': 0.00974529926595541, 'center': -0.0032455444688871404}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EE_r9.xml', 'scale': 0.015158424850746033, 'center': -0.008216661026002492}}, 's4': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_s4.xml', 'scale': 0.006867933306285823, 'center': -0.002979937739167826}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EE_s4.xml', 'scale': 0.013772174062689158, 'center': -0.01275499890321552}}, 'sieie': {'EB': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/weights_finalRegressor_EB_sieie.xml', 'scale': 9.465761735043482e-05, 'center': -1.3066554941611706e-05}, 'EE': {'weights': 'flashgg/Taggers/data/PhoIdInputsCorrections/2018_Legacy/

In [9]:
json.dump(dic,open('{}/xml_files/correctionsSummary{}.json'.format(weightDir,year),'w'),indent=1)